In [1]:
import numpy as np
import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
from packaging import version


from enum import Enum
import fastai
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.utils import midifile
from mido import MidiFile
import os
import gc

In [4]:
print("TensorFlow version: ", tf.__version__)
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


TensorFlow version:  2.9.1
Found GPU at: /device:GPU:0


In [8]:
data_path = "data"
for file in os.listdir(data_path):
    print(file)


x_0.npy
x_1.npy
x_10.npy
x_100.npy
x_101.npy
x_102.npy
x_103.npy
x_104.npy
x_105.npy
x_106.npy
x_107.npy
x_108.npy
x_109.npy
x_11.npy
x_110.npy
x_111.npy
x_112.npy
x_113.npy
x_114.npy
x_115.npy
x_116.npy
x_117.npy
x_118.npy
x_119.npy
x_12.npy
x_120.npy
x_121.npy
x_122.npy
x_123.npy
x_124.npy
x_125.npy
x_126.npy
x_127.npy
x_128.npy
x_129.npy
x_13.npy
x_130.npy
x_131.npy
x_132.npy
x_133.npy
x_134.npy
x_135.npy
x_136.npy
x_137.npy
x_138.npy
x_139.npy
x_14.npy
x_140.npy
x_141.npy
x_142.npy
x_143.npy
x_144.npy
x_145.npy
x_146.npy
x_147.npy
x_148.npy
x_149.npy
x_15.npy
x_150.npy
x_151.npy
x_152.npy
x_153.npy
x_154.npy
x_155.npy
x_156.npy
x_157.npy
x_158.npy
x_159.npy
x_16.npy
x_160.npy
x_161.npy
x_162.npy
x_163.npy
x_164.npy
x_165.npy
x_166.npy
x_167.npy
x_168.npy
x_169.npy
x_17.npy
x_170.npy
x_171.npy
x_172.npy
x_173.npy
x_174.npy
x_175.npy
x_176.npy
x_177.npy
x_178.npy
x_179.npy
x_18.npy
x_180.npy
x_181.npy
x_182.npy
x_183.npy
x_184.npy
x_185.npy
x_186.npy
x_187.npy
x_188.npy
x_189.npy
x_1

In [4]:
%load_ext tensorboard

In [12]:
model = keras.models.load_model("sigmoidCheckpoints")
model.summary()

# this trains 1 epoch
log_dir = "logs/fit/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
x_val = np.load("data/x_200.npy")
y_val = np.load("data/y_200.npy")
for g in range(211,216):
    a = np.load("data/x_" + str(g) + ".npy")
    b = np.load("data/y_" + str(g) + ".npy")
    x_val = np.concatenate((x_val, a))
    y_val = np.concatenate((y_val, b))
for i in range(20):# let's make the last batch the validation (#200-220 inclusive)   
    x = None
    y = None
    gc.collect()
    for j in range(10):
        print(10* i + j)
        new_x = np.load("data/x_" + str(10* i + j) + ".npy")
        new_y = np.load("data/y_" + str(10* i + j) + ".npy")
        if x is None and y is None:
            x = new_x
            y = new_y
        else:
            x = np.concatenate((x, new_x))
            y = np.concatenate((y, new_y))
    print(x.shape)
    model.fit(x, y, validation_data = (x_val, y_val), epochs = 1, callbacks = [tensorboard_callback]) #call evaluate/save with/without improvement
    #model optimization and look at other papers for inspiration


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 85)                8585      
                                                                 
 dense_3 (Dense)             (None, 75)                6450      
                                                                 
 dense_4 (Dense)             (None, 65)                4940      
                                                                 
 dense_5 (Dense)             (None, 35)                2310      
                                                                 
 dense_6 (Dense)             (None, 2)                 7

KeyboardInterrupt: 

In [5]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 30992), started 2 days, 15:13:57 ago. (Use '!kill 30992' to kill it.)

In [6]:
#tensorboard dev upload --logdir \
#   'tensorboard'

SyntaxError: invalid syntax (2121200483.py, line 1)

In [2]:
#todo: make a predict pipeline

model = keras.models.load_model("sigmoidCheckpoints")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 85)                8585      
                                                                 
 dense_3 (Dense)             (None, 75)                6450      
                                                                 
 dense_4 (Dense)             (None, 65)                4940      
                                                                 
 dense_5 (Dense)             (None, 35)                2310      
                                                                 
 dense_6 (Dense)             (None, 2)                 7

In [16]:
def predict_music(model, input_vector, num):
    normalized = input_vector / 311
    for i in range(num):
        predict = model.predict(np.reshape(normalized[-100:], (1,100)), verbose = 0)
        normalized = np.append(normalized, predict)
    
    result = np.rint(normalized * 311)
    # edits to prediction
    for i in range(100, len(result)):
        if abs(result[i] - 8) < 10:
            print("change made to " + str(result[i]))
            result[i] = 8
    return result


In [20]:
data = np.load("data.npy", allow_pickle = True)
test = data[2][0:100]
def midi_predict(model, test):
    test_midi = idxenc2stream(test.astype("int"), MusicVocab.create())
    test_midi.write('midi',"input.mid")
    res = predict_music(model, test, 100)
    print(res[-40:])
    output = idxenc2stream(res.astype("int"), MusicVocab.create())
    output.write('midi',"output.mid")
    return res.astype("int")
array = midi_predict(model, test)
for elem in array:
    print(elem, end = " ")

change made to 17.0
change made to 15.0
change made to 12.0
change made to 11.0
change made to 13.0
change made to 11.0
change made to 13.0
change made to 13.0
change made to 15.0
change made to 16.0
change made to 14.0
change made to 14.0
change made to 13.0
change made to 13.0
change made to 13.0
change made to 12.0
change made to 12.0
change made to 12.0
change made to 12.0
change made to 13.0
change made to 14.0
change made to 14.0
change made to 15.0
change made to 15.0
change made to 16.0
change made to 16.0
[ 78. 140.  71. 140.   8. 140.  78. 140.  71. 140.  18. 140.  79. 140.  72. 140.  18. 140.  79. 140.  73. 140.  19.
 140.  79. 140.  74. 140.  22. 140.  60. 141.  24. 140.  27. 140.  19. 140.  76. 140.]
0 1 80 145 64 139 8 139 59 139 8 139 64 139 8 139 68 139 8 139 71 145 8 139 76 139 8 139 71 138 8 138 76 138 8 139 80 138 8 138 85 145 64 139 8 139 59 139 8 139 64 139 8 139 68 139 8 139 87 141 71 145 8 141 88 141 8 141 76 143 64 139 8 139 59 139 8 139 64 139 8 139 78 139 68 1

In [40]:
# ssh -i "C:\Users\drago\Downloads\ssh-key-2022-10-25.key" ubuntu@150.136.137.227
# scp -i "C:\Users\drago\Downloads\ssh-key-2022-10-25.key" ubuntu@150.136.137.227:/home/ubuntu/projectpathing/updatedCheckpoints C:\Users\drago\Downloads
print(test)
print(test[-100:])

[  0   1   8 153  74 140  69 140  66 140   8 138  62 138  57 142   8 139
  62 139   8 138  74 139  73 138  69 139  66 139   8 139  62 138   8 138
  74 141  69 142  66 139   8 138  62 138  57 144   8 138  66 138   8 138
  62 141   8 138  66 138   8 138  74 140  69 138  66 139   8 138  69 138
   8 139  69 138  66 138   8 138  73 140  69 142  64 140  57 144   8 139
  61 140   8 139  76 140  64 140   8 138]
[  0   1   8 153  74 140  69 140  66 140   8 138  62 138  57 142   8 139
  62 139   8 138  74 139  73 138  69 139  66 139   8 139  62 138   8 138
  74 141  69 142  66 139   8 138  62 138  57 144   8 138  66 138   8 138
  62 141   8 138  66 138   8 138  74 140  69 138  66 139   8 138  69 138
   8 139  69 138  66 138   8 138  73 140  69 142  64 140  57 144   8 139
  61 140   8 139  76 140  64 140   8 138]


In [14]:
print(y.shape)

(772051, 2)


In [36]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_105 (Dense)           (None, 100)               10100     
                                                                 
 dense_106 (Dense)           (None, 100)               10100     
                                                                 
 dense_107 (Dense)           (None, 85)                8585      
                                                                 
 dense_108 (Dense)           (None, 75)                6450      
                                                                 
 dense_109 (Dense)           (None, 65)                4940      
                                                                 
 dense_110 (Dense)           (None, 35)                2310      
                                                                 
 dense_111 (Dense)           (None, 2)               